In [5]:
#!pip install -qU langchain_experimental langchain_openai langchain_community langchain ragas chromadb langchain-groq fastembed pypdf openai konlpy sentence-transformers python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
#!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, SpacyTextSplitter, NLTKTextSplitter, split_text_on_tokens, SentenceTransformersTokenTextSplitter, LatexTextSplitter
from langchain.text_splitter import PythonCodeTextSplitter, KonlpyTextSplitter, ElementType

from langchain.document_loaders import PyPDFLoader
import torch
import re
from docx import Document as worddoc

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents.base import Document

import PyPDF2
import pandas as pd
from datasets import Dataset

In [19]:


class TextChunker:
    def __init__(self):
        pass

    def extract_text_from_pdf(self, path = "/"):
        text = ""
        loader = PyPDFLoader(path)
        documents = loader.load()
        for doc in documents:
            text = text + "\n" + str(doc.page_content)
        metadata = self.get_pdf_metadata(path)
        return text,documents, metadata

    def char_count_chunking_with_overlap(self, text, chunk_size=200, chunk_overlap=50, splitter_type = "CharacterTextSplitter"):
        # Instantiate the CharacterTextSplitter class

        text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        if splitter_type ==  "RecursiveCharacterTextSplitter":# Instantiate the RecursiveCharacterTextSplitter class
           text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        # Create documents using the text splitter
        docs =[]
        chunks = []
        if isinstance(text, str):
          docs = text_splitter.create_documents([text])
          chunks = [doc.page_content for doc in docs]
        elif isinstance(text, list):
          try:
            docs = text_splitter.create_documents(text)
            chunks = [doc.page_content for doc in docs]
          except:
            chunks = text_splitter.split_documents(text)
        return chunks

    #Nonoverlap
    def char_count_chunking_with_nonoveralp(self, text, chunk_size=200, splitter_type = "CharacterTextSplitter"):
       # Instantiate the CharacterTextSplitter class
        text_splitter = CharacterTextSplitter(chunk_size=chunk_size)

        if splitter_type ==  "RecursiveCharacterTextSplitter":# Instantiate the RecursiveCharacterTextSplitter clas
           text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(chunk_size=chunk_size)
        # Create documents using the text splitter
        docs =[]
        chunks = []
        if isinstance(text, str):
          docs = text_splitter.create_documents([text])
          chunks = [doc.page_content for doc in docs]
        elif isinstance(text, list):
          try:
            docs = text_splitter.create_documents(text)
            chunks = [doc.page_content for doc in docs]
          except:
            chunks = text_splitter.split_documents(text)

        return chunks

    #chunking
    def char_count_chunking_with_custom_delimiter(self, text, chunk_size=200, chunk_overlap=50, delimiter="\n\n",splitter_type = "CharacterTextSplitter"):
        # Instantiate the CharacterTextSplitter class
        text_splitter = CharacterTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        if splitter_type ==  "RecursiveCharacterTextSplitter":# Instantiate the RecursiveCharacterTextSplitter class
           text_splitter = RecursiveCharacterTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "NLTKTextSplitter":
           text_splitter = NLTKTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SentenceTransformersTokenTextSplitter":
           text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "LatexTextSplitter":
           text_splitter = LatexTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "PythonCodeTextSplitter":
           text_splitter = PythonCodeTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "KonlpyTextSplitter":
           text_splitter = KonlpyTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        # Create documents using the text splitter
        docs =[]
        chunks = []
        if isinstance(text, str):
          docs = text_splitter.create_documents([text])
          chunks = [doc.page_content for doc in docs]
        else:
          try:
            chunks = text_splitter.split_documents(text)
          except:
            docs = text_splitter.create_documents(text)
            chunks = [doc.page_content for doc in docs]

        return chunks

    #Semantic chunking
    def semantic_section_chunking(self, documents , text_embedding_model_name,  breakpoint_threshold_type = "percentile"):
        embed_model = FastEmbedEmbeddings(model_name = text_embedding_model_name)
        semantic_chunker = SemanticChunker(embed_model, breakpoint_threshold_type=breakpoint_threshold_type)
        try:
          semantic_chunks = semantic_chunker.create_documents([d.page_content for d in documents])
        except:
          semantic_chunks = semantic_chunker.create_documents([d for d in markdown_splits])
        return semantic_chunks

    #Markdown HeaderTextSplitter
    def markdown_header_textsplitter(self, text, headers_to_split_on, strip_headers):
        from langchain.text_splitter import MarkdownHeaderTextSplitter

        markdown_document = text
        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on,  strip_headers=strip_headers)
        md_header_splits = markdown_splitter.split_text(markdown_document)
        return md_header_splits

    #Markdown TextSplitter
    def markdown_textsplitter(self, text):
        from langchain.text_splitter import MarkdownTextSplitter
        splitter = MarkdownTextSplitter()
        split_text = splitter.split_text(text)
        return split_text

    #ouputs to Doc
    #from docx import Document
    #import re
    def save_as_word_doc(self, split_text, file_path ,heading):
        doc = worddoc()
        doc.add_heading(heading, level=1)
        for i, text_chunk in enumerate(split_text):
            text_chunk= re.sub(r'[\x00-\x1F]+', '', str(text_chunk))
            doc.add_heading(f'Chunk {i + 1}', level=1)
            doc.add_paragraph(text_chunk)
        # Save the document
        doc.save(file_path)
        print("Output saved as doc successfully")

    #Get pdf _metadata
    def get_pdf_metadata(self, pdf_path):
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            metadata = reader.metadata
            return metadata

    #create text dataframe as Langchain Document Schema
    def Create_Hugging_Face_Dataset(self, pdf_path):
        data = {
            'text': [],
            'source': []
        }

        text, documents = self.extract_text_from_pdf(pdf_path)
        metadata = self.get_pdf_metadata(pdf_path)
        source = str(metadata)
        data['text'].append(text)
        data['source'].append(source)
        df = pd.DataFrame(data)
        dataset = Dataset.from_pandas(df)# Convert pandas DataFrame to Hugging Face Dataset
        return dataset, documents

#################################################################################
#Usage
pdf_path =r"/content/drive/MyDrive/chunking/ds (1).pdf"

text_chunker = TextChunker()
text, documents, metadata  = text_chunker.extract_text_from_pdf(path = pdf_path)

In [10]:
#Chunking
markdown_split_text = text_chunker.markdown_textsplitter(text)

SentenceTransformers_chunks = text_chunker.char_count_chunking_with_custom_delimiter(markdown_split_text, chunk_size=200, chunk_overlap=50, splitter_type = "SentenceTransformersTokenTextSplitter")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [51]:
import PyPDF2
from tqdm.notebook import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document as LangchainDocument

def get_pdf_metadata(pdf_path):
    reader = PyPDF2.PdfReader(pdf_path)
    metadata = reader.metadata
    return metadata

def extract_pdf_content_with_metadata(pdf_path: str):
    # Extract the overall PDF metadata
    overall_metadata = get_pdf_metadata(pdf_path)

    # Load the PDF and extract text and page-specific metadata using PyPDFLoader
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    pages_content = []

    for i, doc in enumerate(tqdm(documents, desc="Processing PDF Pages")):
        page_content = doc.page_content
        page_metadata = doc.metadata

        # Combine overall metadata with page-specific metadata
        combined_metadata = {
            "source": pdf_path,
            "page_number": i + 1,
            **overall_metadata,  # Include overall PDF metadata
            **page_metadata  # Include existing page-specific metadata
        }

        pages_content.append((page_content, combined_metadata))

    return pages_content

# Path to your PDF file
pdf_path = r"/content/drive/MyDrive/chunking/ds (1).pdf"

# Extract text and metadata from the PDF
pdf_content = extract_pdf_content_with_metadata(pdf_path)

# Create LangchainDocument objects
RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=text, metadata=metadata) for text, metadata in pdf_content
]

# Print the number of pages and the content of the second page
print(len(RAW_KNOWLEDGE_BASE))
print(RAW_KNOWLEDGE_BASE[1].page_content)
print(RAW_KNOWLEDGE_BASE[1].metadata)


Processing PDF Pages:   0%|          | 0/32 [00:00<?, ?it/s]

32
2
{'source': '/content/drive/MyDrive/chunking/ds (1).pdf', 'page_number': 2, '/Author': '', '/Title': '', '/Subject': '', '/Creator': 'LaTeX with hyperref package', '/Producer': 'pdfTeX-1.40.19', '/Keywords': '', '/CreationDate': "D:20190814093040+02'00'", '/ModDate': "D:20190814093040+02'00'", '/Trapped': '/False', '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0', 'page': 1}


In [59]:
len(RAW_KNOWLEDGE_BASE)

32

In [56]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class.
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # the maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=100,  # the number of characters to overlap between chunks
    add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
    docs_processed += text_splitter.split_documents([doc])

print(len(docs_processed))

91


In [58]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

EMBEDDING_MODEL_NAME = "thenlper/gte-small"
chunk_size = 512
#
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )
#
docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
  docs_processed += text_splitter.split_documents([doc])

print(len(docs_processed))

72


In [61]:

from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    #model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = Chroma.from_documents(docs_processed,
                                                  embedding_model,
                                                  persist_directory="/content/drive/MyDrive/chunking/CRAG",
                                                  collection_name="crag")
#
KNOWLEDGE_VECTOR_DATABASE.persist()
print(len(KNOWLEDGE_VECTOR_DATABASE.get()['documents']))

72


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [67]:
user_query = "what is Kantorovitch formulation?"
print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
print("\n==================================Top document==================================")
print(retrieved_docs[1].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[1].metadata)


Starting retrieval for user_query='what is Kantorovitch formulation?'...

==================================Top document==================================
exists faster combinatorial optimization scheme, the most famous ones being the Hungarian algorithm and
the auction algorithm, which have roughly O(n3) complexity. Section 1.5 details an approximate algorithm,
which is typically faster, and amenable to parallelisation, but do not compute exactly the solution to the
OT problem.
1.4 Duality
The Kantorovich problem (1.11) is a constrained convex minimization problem, and as such, it can be
naturally paired with a so-called dual problem, which is a constrained concave maximization problem. The
following fundamental proposition, which is a special case of Fenchel-Rockafellar duality theory, explains the
relationship between the primal and dual problems.
Proposition 4. One has
LC(a,b) = max
(f,g)∈R(a,b)⟨f,a⟩+⟨g,b⟩ (1.17)
where the set of admissible potentials is
R(a,b)def.={(f,g)∈Rn×Rm;∀(

In [78]:
KNOWLEDGE_VECTOR_DATABASE.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])

72

In [73]:
# Retrieve all documents with their indices

KNOWLEDGE_VECTOR_DATABASE.get()#['documents']


{'ids': ['030740bf-4a69-46b0-a9d4-aef7b5d8ba26',
  '10357603-7f8b-4be0-9387-8382178cb882',
  '11207380-8cb0-43d6-aa1d-9b6fbade2808',
  '2168c2e6-6af9-4adf-934d-9aa3c73d45ba',
  '2258c2ec-9082-4138-a3a3-a56204021ea1',
  '2660fdd6-0e11-4ffd-b6db-685976f1b15c',
  '28b60564-f4af-4fd5-99ec-1c0ca67acae2',
  '2b7514db-5e38-43d1-b5c7-5c79b3604d03',
  '31ddd804-ca4b-4aef-b596-f92ef00150ad',
  '328c9e3e-e060-4d0d-b626-9b66244c7d78',
  '335d1842-30a0-4d99-8cf2-14fe625709c5',
  '3f32ddd8-f0a1-4eda-acad-9e4e61846a81',
  '3fd594d1-9ae0-4589-a47d-2b779b9b3272',
  '44c6dda4-55aa-4e63-844f-8a574ef8a48d',
  '4980c084-eca8-4012-9426-56ac93f4a5c6',
  '57152b3d-9b72-475d-b4bc-c9fd889702c2',
  '583008e8-e60b-402b-9c83-cbfc26fda394',
  '5ae578fe-564d-4468-8a14-44b1005a3dac',
  '5cf20c30-ee92-4e0f-b086-872a4858a8dd',
  '5d3da48b-2edd-4da0-bae1-33e01344e2aa',
  '618da788-1e6c-4558-95c9-798064a3bd22',
  '6488d160-9213-4ab0-bb6d-06af8af00d7f',
  '6dd2afc8-5379-4884-b302-daf2bf86d734',
  '6f15fcef-fd37-4579-8836-

In [75]:
KNOWLEDGE_VECTOR_DATABASE.get()["metadatas"]

[{'/Author': '',
  '/CreationDate': "D:20190814093040+02'00'",
  '/Creator': 'LaTeX with hyperref package',
  '/Keywords': '',
  '/ModDate': "D:20190814093040+02'00'",
  '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0',
  '/Producer': 'pdfTeX-1.40.19',
  '/Subject': '',
  '/Title': '',
  '/Trapped': '/False',
  'page': 14,
  'page_number': 15,
  'source': '/content/drive/MyDrive/chunking/ds (1).pdf',
  'start_index': 0},
 {'/Author': '',
  '/CreationDate': "D:20190814093040+02'00'",
  '/Creator': 'LaTeX with hyperref package',
  '/Keywords': '',
  '/ModDate': "D:20190814093040+02'00'",
  '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0',
  '/Producer': 'pdfTeX-1.40.19',
  '/Subject': '',
  '/Title': '',
  '/Trapped': '/False',
  'page': 13,
  'page_number': 14,
  'source': '/content/drive/MyDrive/chunking/ds (1).pdf',
  'start_index': 0},
 {'/Author': '',
  '/CreationDat

In [87]:
# Assuming KNOWLEDGE_VECTOR_DATABASE.get() returns a dictionary with keys 'ids',  'metadatas', 'documents'

knowledge_data = KNOWLEDGE_VECTOR_DATABASE.get()

organized_data = []

for i in range(len(knowledge_data['ids'])):
    data_dict = {
        'id': knowledge_data['ids'][i],
        'metadata': knowledge_data['metadatas'][i],
        'document': knowledge_data['documents'][i]
    }
    organized_data.append(data_dict)




In [88]:
 organized_data

[{'id': '030740bf-4a69-46b0-a9d4-aef7b5d8ba26',
  'metadata': {'/Author': '',
   '/CreationDate': "D:20190814093040+02'00'",
   '/Creator': 'LaTeX with hyperref package',
   '/Keywords': '',
   '/ModDate': "D:20190814093040+02'00'",
   '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0',
   '/Producer': 'pdfTeX-1.40.19',
   '/Subject': '',
   '/Title': '',
   '/Trapped': '/False',
   'page': 14,
   'page_number': 15,
   'source': '/content/drive/MyDrive/chunking/ds (1).pdf',
   'start_index': 0},
  'document': '\x00\x00↵Figure 1.8: 1-D optimal couplings: each arrow xi→yjindicate a non-zero Pi,jin the optimal coupling.\nTop: empirical measures with same number of points (optimal matching). Bottom: generic case. This\ncorresponds to monotone rearrangements, if xi⩽xi′are such that Pi,j̸= 0,Pi′,j′̸= 0, then necessarily\nyj⩽yj′.\nRemark 7 (1-D case – Empirical measures) .HereX=R. Assuming α=1\nn∑n\ni=1δxiandβ=1\nn∑n\nj=1δyj,\nand assum